In [10]:
import asyncio
from mongo_db_handler import MongoDBHandler
async def perform_searches():
    db_url = 'your_mongodb_url'
    db_name = 'your_db_name'
    handler = MongoDBHandler(db_url, db_name)

    # Example queries
    queries = ["search_term_1", "search_term_2", "search_term_3"]

    # Perform searches concurrently using asyncio.gather
    tasks = [handler.search(query) for query in queries]
    results = await asyncio.gather(*tasks)

    # Print or process the results
    for query, result in zip(queries, results):
        print(f"Search results for '{query}': {result}")

if __name__ == "__main__":
    asyncio.run(perform_searches())

RuntimeError: asyncio.run() cannot be called from a running event loop

## Average Time by simple search from  mongo_db

In [18]:
import time
from mongo_db_handler import MongoDBHandler
mongo_handler = MongoDBHandler(collection_name = "pdf_collection", db_name="pdf_engine")
mongo_handler.count_entries()
average_time = 0
for i in range(50):
    print(i, end=':')
    start_time = time.time()
    res = mongo_handler.search("physics")
    print(len(list(res)))
    stop_time = time.time()
    duration = stop_time - start_time
    average_time += duration
average_time /= 15
average_time    

Number of entries in the collection: 51471
0:47
1:47
2:47
3:47
4:47
5:47
6:47
7:47
8:47
9:47
10:47
11:47
12:47
13:47
14:47
15:47
16:47
17:47
18:47
19:47
20:47
21:47
22:47
23:47
24:47
25:47
26:47
27:47
28:47
29:47
30:47
31:47
32:47
33:47
34:47
35:47
36:47
37:47
38:47
39:47
40:47
41:47
42:47
43:47
44:47
45:47
46:47
47:47
48:47
49:47


5.98073132832845

## Average Time by search_by_id from mongodb

In [19]:
import json
import time

with open('/home/anon/ioee/pdf_engine_search_result_physics.json', 'r') as f:
    data = json.load(f)

ids = [d['id'] for d in data]


from mongo_db_handler import MongoDBHandler
mongo_handler = MongoDBHandler(collection_name = "pdf_collection", db_name="pdf_engine")
mongo_handler.count_entries()
average_time = 0
for i in range(50):
    print(i, end=':')
    start_time = time.time()
    res = mongo_handler.get_by_ids(ids)
    print(len(list(res)))
    stop_time = time.time()
    duration = stop_time - start_time
    average_time += duration
average_time /= 15
average_time

Number of entries in the collection: 51471
0:47
1:47
2:47
3:47
4:47
5:47
6:47
7:47
8:47
9:47
10:47
11:47
12:47
13:47
14:47
15:47
16:47
17:47
18:47
19:47
20:47
21:47
22:47
23:47
24:47
25:47
26:47
27:47
28:47
29:47
30:47
31:47
32:47
33:

## Average Time by entire-search-result-in-redis from Redis insertation

In [ ]:
import redis

r = redis.Redis(
host='redis-19089.c212.ap-south-1-1.ec2.cloud.redislabs.com',
port=19089,
password='uUzG92fuoNbpDa2922BTIFnpXyPSgIrO')


In [ ]:
import redis
import json

with open('/home/anon/ioee/pdf_engine_search_result_physics.json', 'r') as f:
    data = json.load(f)

r.set('physics', json.dumps(data))
# True




import json
import time

ids = [d['id'] for d in data]

average_time = 0
for i in range(50):
    print(i, end=':')
    start_time = time.time()
    res = json.loads(r.get('physics'))
    print(len(list(res)))
    stop_time = time.time()
    duration = stop_time - start_time
    average_time += duration
average_time /= 15
average_time



# b'bar'

0:47
1:47
2:47
3:47
4:47
5:47
6:47
7:47
8:47
9:47
10:47
11:47
12:47
13:47
14:47
15:47
16:47
17:47
18:47
19:47
20:47
21:47
22:47
23:47
24:47
25:47
26:47
27:47
28:47
29:47
30:47
31:47
32:47
33:47
34:47
35:47
36:47
37:47
38:47
39:47
40:47
41:47
42:47
43:47
44:47
45:47
46:47
47:47
48:47
49:47


3.6866156101226806

## Average Time by ids-from-redis, data-froom-mongo

In [17]:
import redis
import json

with open('/home/anon/ioee/pdf_engine_search_result_physics.json', 'r') as f:
    data = json.load(f)

r.set('physics_ids', json.dumps([d['id'] for d in data]))

from mongo_db_handler import MongoDBHandler
mongo_handler = MongoDBHandler(collection_name = "pdf_collection", db_name="pdf_engine")
mongo_handler.count_entries()
average_time = 0
for i in range(50):
    print(i, end=':')
    start_time = time.time()
    ids = json.loads(r.get('physics_ids'))
    res = mongo_handler.get_by_ids(ids)
    print(len(list(res)))
    stop_time = time.time()
    duration = stop_time - start_time
    average_time += duration
average_time /= 15
average_time

Number of entries in the collection: 51471
0:47
1:47
2:47
3:47
4:47
5:47
6:47
7:47
8:47
9:47
10:47
11:47
12:47
13:47
14:47
15:47
16:47
17:47
18:47
19:47
20:47
21:47
22:47
23:47
24:47
25:47
26:47
27:47
28:47
29:47
30:47
31:47
32:47
33:47
34:47
35:47
36:47
37:47
38:47
39:47
40:47
41:47
42:47
43:47
44:47
45:47
46:47
47:47
48:47
49:47


2.4366972128550213

In [9]:
r.keys()


[b'foo']